# Фабричный метод (Factory Method)

Порождающий паттерн проектирования, который определяет общий интерфейс для создания объектов в суперклассе, позволяя подклассам изменять тип создаваемых объектов.

Фабричный метод предлагает создавать объекты не напрямую через оператор new, а через вызов специального фабричного метода. Объекты по-прежнему создаются через new, но делает это фабричный метод, который можно переопределить в подклассах.


❌ Было:  product = new ConcreteProduct()
✅ Стало: product = creator.factory_method()

Когда применять
✅ Типы объектов неизвестны заранее — код должен работать с объектами, которые определяются во время выполнения.


✅ Нужно расширять библиотеку/фреймворк — пользователи могут добавлять новые типы продуктов через подклассы.


✅ Хотите экономить ресурсы — фабричный метод может возвращать объекты из кэша, а не создавать новые каждый раз.

✅ Соблюдение принципа Open/Closed — добавление новых продуктов не требует изменения существующего кода.


In [ ]:
%%writefile factory.cpp

// ============================================
// ПОДКЛЮЧЕНИЕ БИБЛИОТЕК
// ============================================
#include <iostream>      // Для ввода/вывода (std::cout, std::endl)
#include <memory>        // Для умных указателей (std::unique_ptr)
#include <string>        // Для работы со строками (std::string)

// ============================================
// 1. ПРОДУКТ (Product) — интерфейс создаваемых объектов
// ============================================
// Это абстрактный класс, который определяет общий интерфейс
// для всех продуктов, которые будет создавать фабрика
class Product {
public:
    // Виртуальный деструктор — обязателен для полиморфизма!
    // Позволяет корректно удалять объекты через указатель на базовый класс
    virtual ~Product() = default;

    // Чисто виртуальный метод — делает класс абстрактным
    // = 0 означает, что метод должен быть реализован в наследниках
    // const — метод не изменяет состояние объекта
    virtual std::string operation() const = 0;
};

// ============================================
// 2. КОНКРЕТНЫЕ ПРОДУКТЫ (ConcreteProduct)
// ============================================
// Это конкретные реализации интерфейса Product
// Каждый продукт делает что-то своё в методе operation()

// Продукт №1
class ConcreteProduct1 : public Product {
public:
    // override — ключевое слово, которое проверяет,
    // что мы действительно переопределяем виртуальный метод
    std::string operation() const override {
        return "{Результат ConcreteProduct1}";
    }
};

// Продукт №2
class ConcreteProduct2 : public Product {
public:
    std::string operation() const override {
        return "{Результат ConcreteProduct2}";
    }
};

// ============================================
// 3. СОЗДАТЕЛЬ (Creator) — содержит фабричный метод
// ============================================
// Это абстрактный класс, который объявляет фабричный метод
// и содержит бизнес-логику, использующую продукты
class Creator {
public:
    // Виртуальный деструктор для корректного удаления наследников
    virtual ~Creator() = default;

    // ========================================
    // ФАБРИЧНЫЙ МЕТОД — сердце паттерна!
    // ========================================
    // pure virtual (= 0) — подклассы ОБЯЗАНЫ реализовать этот метод
    // const — метод не изменяет состояние создателя
    // Возвращает умный указатель на Product
    virtual std::unique_ptr<Product> factory_method() const = 0;

    // Бизнес-логика, которая использует продукт
    // Не знает о конкретных классах продуктов — работает через интерфейс
    std::string some_operation() const {
        // Вызываем фабричный метод для создания продукта
        // Какой именно продукт создастся — решают подклассы
        auto product = factory_method();

        // Используем продукт через общий интерфейс
        return "Creator: Работаем с " + product->operation();
    }
};

// ============================================
// 4. КОНКРЕТНЫЕ СОЗДАТЕЛИ (ConcreteCreator)
// ============================================
// Эти классы решают, какой именно продукт создавать
// Каждый возвращает свой тип продукта

// Создатель для Продукта №1
class ConcreteCreator1 : public Creator {
public:
    // Переопределяем фабричный метод
    // Возвращаем конкретный тип продукта
    std::unique_ptr<Product> factory_method() const override {
        // std::make_unique — современный способ создания объектов (C++14+)
        // Автоматически управляет памятью — не нужно удалять вручную
        return std::make_unique<ConcreteProduct1>();
    }
};

// Создатель для Продукта №2
class ConcreteCreator2 : public Creator {
public:
    std::unique_ptr<Product> factory_method() const override {
        return std::make_unique<ConcreteProduct2>();
    }
};

// ============================================
// 5. КЛИЕНТСКИЙ КОД
// ============================================
// Этот код работает с абстракциями (Creator, Product)
// Не зависит от конкретных классов — легко расширять
void client_code(const Creator& creator) {
    // Вызываем бизнес-логику создателя
    // Какой продукт внутри — не важно, работает через интерфейс
    std::cout << creator.some_operation() << std::endl;
}

// ============================================
// 6. ТОЧКА ВХОДА (main)
// ============================================
int main() {
    // Заголовок для первого теста
    std::cout << "App: Запущен с ConcreteCreator1" << std::endl;

    // Создаём конкретного создателя №1
    // Он будет создавать ConcreteProduct1
    ConcreteCreator1 creator1;

    // Передаём создателя в клиентский код
    client_code(creator1);

    // Заголовок для второго теста
    std::cout << "App: Запущен с ConcreteCreator2" << std::endl;

    // Создаём конкретного создателя №2
    // Он будет создавать ConcreteProduct2
    ConcreteCreator2 creator2;

    // Передаём создателя в клиентский код
    client_code(creator2);

    // Успешное завершение программы
    return 0;
}

Overwriting factory.cpp


In [ ]:
!g++ -std=c++17 -Wall -Wextra -o factory factory.cpp
!./factory

App: Запущен с ConcreteCreator1
Creator: Работаем с {Результат ConcreteProduct1}
App: Запущен с ConcreteCreator2
Creator: Работаем с {Результат ConcreteProduct2}


расширить код (добавить Product3)

In [ ]:
%%writefile factory_extended.cpp

// ============================================
// ПОДКЛЮЧЕНИЕ БИБЛИОТЕК
// ============================================
#include <iostream>
#include <memory>
#include <string>

// ============================================
// 1. ПРОДУКТ (интерфейс)
// ============================================
class Product {
public:
    virtual ~Product() = default;
    virtual std::string operation() const = 0;
};

// ============================================
// 2. КОНКРЕТНЫЕ ПРОДУКТЫ
// ============================================
class ConcreteProduct1 : public Product {
public:
    std::string operation() const override {
        return "{Результат ConcreteProduct1}";
    }
};

class ConcreteProduct2 : public Product {
public:
    std::string operation() const override {
        return "{Результат ConcreteProduct2}";
    }
};

// 🆕 НОВЫЙ: Продукт №3
class ConcreteProduct3 : public Product {
public:
    std::string operation() const override {
        return "{Результат ConcreteProduct3}";
    }
};

// ============================================
// 3. СОЗДАТЕЛЬ (абстрактный)
// ============================================
class Creator {
public:
    virtual ~Creator() = default;
    virtual std::unique_ptr<Product> factory_method() const = 0;

    std::string some_operation() const {
        auto product = factory_method();
        return "Creator: Работаем с " + product->operation();
    }
};

// ============================================
// 4. КОНКРЕТНЫЕ СОЗДАТЕЛИ
// ============================================
class ConcreteCreator1 : public Creator {
public:
    std::unique_ptr<Product> factory_method() const override {
        return std::make_unique<ConcreteProduct1>();
    }
};

class ConcreteCreator2 : public Creator {
public:
    std::unique_ptr<Product> factory_method() const override {
        return std::make_unique<ConcreteProduct2>();
    }
};

// 🆕 НОВЫЙ: Создатель №3
class ConcreteCreator3 : public Creator {
public:
    std::unique_ptr<Product> factory_method() const override {
        return std::make_unique<ConcreteProduct3>();
    }
};

// ============================================
// 5. КЛИЕНТСКИЙ КОД
// ============================================
void client_code(const Creator& creator) {
    std::cout << creator.some_operation() << std::endl;
}

// ============================================
// 6. ТОЧКА ВХОДА
// ============================================
int main() {
    std::cout << "=== Тест Factory Method ===" << std::endl << std::endl;

    // Тест 1: Создатель №1
    std::cout << "App: Запущен с ConcreteCreator1:" << std::endl;
    ConcreteCreator1 creator1;
    client_code(creator1);
    std::cout << std::endl;

    // Тест 2: Создатель №2
    std::cout << "App: Запущен с ConcreteCreator2:" << std::endl;
    ConcreteCreator2 creator2;
    client_code(creator2);
    std::cout << std::endl;

    // 🆕 Тест 3: Создатель №3 (НОВЫЙ!)
    std::cout << "App: Запущен с ConcreteCreator3:" << std::endl;
    ConcreteCreator3 creator3;
    client_code(creator3);

    return 0;
}

Overwriting factory_extended.cpp


In [ ]:
# Компиляция нового файла
!g++ -std=c++17 -Wall -Wextra -o factory_extended factory_extended.cpp

# Запуск программы
!./factory_extended

=== Тест Factory Method ===

App: Запущен с ConcreteCreator1:
Creator: Работаем с {Результат ConcreteProduct1}

App: Запущен с ConcreteCreator2:
Creator: Работаем с {Результат ConcreteProduct2}

App: Запущен с ConcreteCreator3:
Creator: Работаем с {Результат ConcreteProduct3}


# Абстрактная фабрика (Abstract Factory)

Порождающий паттерн, который позволяет создавать семейства связанных объектов без привязки к их конкретным классам создаваемых объектов. создаёт СЕМЕЙСТВА связанных объектов, гарантируя их совместимость между собой.        

❌ Factory Method:  creator.factory_method() → Product

✅ Abstract Factory: factory.createButton() + factory.createCheckbox() → Семейство UI

Гарантия совместимости-Все продукты из одной фабрики точно работают вместе

Изоляция кода создания-
Клиент не знает о конкретных классах

Легкая замена семейства-Поменяли фабрику — поменялся весь UI

Принцип Open/Closed-Новые семейства добавляются без изменения клиента

In [ ]:
%%writefile abstract_factory.cpp

// ============================================
// ПОДКЛЮЧЕНИЕ БИБЛИОТЕК
// ============================================
#include <iostream>
#include <memory>
#include <string>

// ============================================
// 🎨 ПРОДУКТЫ: Семейство A (Кнопки)
// ============================================
// Абстрактный интерфейс для всех кнопок
class AbstractButton {
public:
    virtual ~AbstractButton() = default;

    // Каждый продукт должен уметь отрисовываться
    virtual void paint() const = 0;

    // И обрабатывать нажатия
    virtual std::string get_name() const = 0;
};

// ============================================
// 🎨 ПРОДУКТЫ: Семейство B (Чекбоксы)
// ============================================
// Абстрактный интерфейс для всех чекбоксов
class AbstractCheckbox {
public:
    virtual ~AbstractCheckbox() = default;

    virtual void paint() const = 0;
    virtual std::string get_name() const = 0;
};

// ============================================
// 🪟 КОНКРЕТНЫЕ ПРОДУКТЫ: Семейство Windows
// ============================================

// Windows-кнопка
class WindowsButton : public AbstractButton {
public:
    void paint() const override {
        std::cout << "🪟 Отрисовка кнопки в стиле Windows" << std::endl;
    }

    std::string get_name() const override {
        return "WindowsButton";
    }
};

// Windows-чекбокс
class WindowsCheckbox : public AbstractCheckbox {
public:
    void paint() const override {
        std::cout << "🪟 Отрисовка чекбокса в стиле Windows" << std::endl;
    }

    std::string get_name() const override {
        return "WindowsCheckbox";
    }
};

// ============================================
// 🍎 КОНКРЕТНЫЕ ПРОДУКТЫ: Семейство Mac
// ============================================

// Mac-кнопка
class MacButton : public AbstractButton {
public:
    void paint() const override {
        std::cout << "🍎 Отрисовка кнопки в стиле Mac" << std::endl;
    }

    std::string get_name() const override {
        return "MacButton";
    }
};

// Mac-чекбокс
class MacCheckbox : public AbstractCheckbox {
public:
    void paint() const override {
        std::cout << "🍎 Отрисовка чекбокса в стиле Mac" << std::endl;
    }

    std::string get_name() const override {
        return "MacCheckbox";
    }
};

// ============================================
// 🏭 АБСТРАКТНАЯ ФАБРИКА (интерфейс)
// ============================================
// Объявляет методы для создания ВСЕХ продуктов семейства
class AbstractFactory {
public:
    virtual ~AbstractFactory() = default;

    // 🎨 Методы для создания семейства продуктов
    virtual std::unique_ptr<AbstractButton> create_button() const = 0;
    virtual std::unique_ptr<AbstractCheckbox> create_checkbox() const = 0;
};

// ============================================
// 🪟 КОНКРЕТНАЯ ФАБРИКА: Windows
// ============================================
// Создаёт ВСЕ продукты в стиле Windows
class WindowsFactory : public AbstractFactory {
public:
    std::unique_ptr<AbstractButton> create_button() const override {
        return std::make_unique<WindowsButton>();
    }

    std::unique_ptr<AbstractCheckbox> create_checkbox() const override {
        return std::make_unique<WindowsCheckbox>();
    }
};

// ============================================
// 🍎 КОНКРЕТНАЯ ФАБРИКА: Mac
// ============================================
// Создаёт ВСЕ продукты в стиле Mac
class MacFactory : public AbstractFactory {
public:
    std::unique_ptr<AbstractButton> create_button() const override {
        return std::make_unique<MacButton>();
    }

    std::unique_ptr<AbstractCheckbox> create_checkbox() const override {
        return std::make_unique<MacCheckbox>();
    }
};

// ============================================
// 🖥️ КЛИЕНТСКИЙ КОД: Приложение
// ============================================
// Работает ТОЛЬКО с абстракциями — не знает о конкретных фабриках
class Application {
private:
    std::unique_ptr<AbstractButton> button_;
    std::unique_ptr<AbstractCheckbox> checkbox_;

public:
    // 🔑 Конструктор принимает ЛЮБУЮ фабрику!
    // Это ключевая точка гибкости паттерна
    explicit Application(const AbstractFactory& factory) {
        // Создаём ВСЕ продукты через одну фабрику
        // Гарантируем, что они из ОДНОГО семейства!
        button_ = factory.create_button();
        checkbox_ = factory.create_checkbox();
    }

    // Бизнес-логика приложения
    void render() const {
        std::cout << "\n=== Отрисовка интерфейса ===" << std::endl;

        // Используем продукты через общий интерфейс
        button_->paint();
        checkbox_->paint();

        std::cout << "Созданы: " << button_->get_name()
                  << " + " << checkbox_->get_name() << std::endl;
    }
};

// ============================================
// 🚀 ТОЧКА ВХОДА
// ============================================
int main() {
    std::cout << "🎨 Abstract Factory Pattern Demo" << std::endl;
    std::cout << "================================" << std::endl;

    // ────────────────────────────────────────
    // Сценарий 1: Запуск под Windows
    // ────────────────────────────────────────
    std::cout << "\n🔹 ЗАПУСК ПОД WINDOWS:" << std::endl;
    {
        // Создаём фабрику для Windows
        WindowsFactory win_factory;

        // Передаём фабрику в приложение
        // Приложение НЕ ЗНАЕТ, что это WindowsFactory!
        Application app(win_factory);

        // Приложение создаёт совместимые продукты
        app.render();
    }

    // ────────────────────────────────────────
    // Сценарий 2: Запуск под Mac
    // ────────────────────────────────────────
    std::cout << "\n🔹 ЗАПУСК ПОД MAC:" << std::endl;
    {
        // Создаём фабрику для Mac
        MacFactory mac_factory;

        // Тот же код приложения, ДРУГАЯ фабрика!
        Application app(mac_factory);

        // Приложение создаёт продукты в стиле Mac
        app.render();
    }

    // ────────────────────────────────────────
    // Сценарий 3: Динамический выбор фабрики
    // ────────────────────────────────────────
    std::cout << "\n🔹 ДИНАМИЧЕСКИЙ ВЫБОР ОС:" << std::endl;
    {
        // Имитация определения ОС во время выполнения
        bool is_windows = true;  // Можно получить из config/argv

        // Выбираем фабрику динамически
        std::unique_ptr<AbstractFactory> factory;
        if (is_windows) {
            factory = std::make_unique<WindowsFactory>();
        } else {
            factory = std::make_unique<MacFactory>();
        }

        // Приложение работает с любой фабрикой
        Application app(*factory);
        app.render();
    }

    std::cout << "\n✅ Программа завершена!" << std::endl;
    return 0;
}

Writing abstract_factory.cpp


In [ ]:
# Компиляция с предупреждениями
!g++ -std=c++17 -Wall -Wextra -o abstract_factory abstract_factory.cpp

# Запуск программы
!./abstract_factory

🎨 Abstract Factory Pattern Demo

🔹 ЗАПУСК ПОД WINDOWS:

=== Отрисовка интерфейса ===
🪟 Отрисовка кнопки в стиле Windows
🪟 Отрисовка чекбокса в стиле Windows
Созданы: WindowsButton + WindowsCheckbox

🔹 ЗАПУСК ПОД MAC:

=== Отрисовка интерфейса ===
🍎 Отрисовка кнопки в стиле Mac
🍎 Отрисовка чекбокса в стиле Mac
Созданы: MacButton + MacCheckbox

🔹 ДИНАМИЧЕСКИЙ ВЫБОР ОС:

=== Отрисовка интерфейса ===
🪟 Отрисовка кнопки в стиле Windows
🪟 Отрисовка чекбокса в стиле Windows
Созданы: WindowsButton + WindowsCheckbox

✅ Программа завершена!


# Строитель (Builder)

Порождающий паттерн, который позволяет создавать сложные объекты пошагово, отделяя процесс конструирования от представления объекта.

Сложные объекты с этапами
Пошаговое конструирование + разные представления

разделяет создание сложного объекта на этапы:
1️⃣ Создать пустой объект
2️⃣ Пошагово настроить части (setWheels, setEngine, ...)
3️⃣ Получить готовый продукт

🔑 Разные строители могут создавать разные представления одного и того же объекта                  

Builder = "Пошаговый конструктор"

🔹 Product—ЧТО строим (сложный объект)
🔹 Builder     — КАК строим (интерфейс шагов)
🔹 ConcreteBuilder — КОНКРЕТНЫЕ версии (Gaming, Office...)
🔹 Director    — УПРАВЛЯЕТ процессом (опционально)
🔹 Client      — ВЫБИРАЕТ строителя и запускает сборку
🎯Ключевая идея: Один процесс → разные результаты!  

In [ ]:
%%writefile builder.cpp

// ============================================
// ПОДКЛЮЧЕНИЕ БИБЛИОТЕК
// ============================================
#include <iostream>
#include <memory>
#include <string>
#include <vector>
#include <iomanip>

// ============================================
// 🖥️ ПРОДУКТ: Компьютер (сложный объект)
// ============================================
// Объект, который мы будем собирать пошагово
class Computer {
private:
    // Части компьютера
    std::string cpu_;
    std::string ram_;
    std::string storage_;
    std::string gpu_;
    bool has_wifi_;
    bool has_bluetooth_;
    std::vector<std::string> peripherals_;  // Доп. устройства

public:
    // 🔑 Конструктор приватный — создаём только через Builder!
    Computer() : has_wifi_(false), has_bluetooth_(false) {}

    // === Сеттеры (вызывает только Builder) ===
    void set_cpu(const std::string& cpu) { cpu_ = cpu; }
    void set_ram(const std::string& ram) { ram_ = ram; }
    void set_storage(const std::string& storage) { storage_ = storage; }
    void set_gpu(const std::string& gpu) { gpu_ = gpu; }
    void set_wifi(bool wifi) { has_wifi_ = wifi; }
    void set_bluetooth(bool bt) { has_bluetooth_ = bt; }
    void add_peripheral(const std::string& peripheral) {
        peripherals_.push_back(peripheral);
    }

    // === Геттеры для отображения ===
    const std::string& get_cpu() const { return cpu_; }
    const std::string& get_ram() const { return ram_; }
    const std::string& get_storage() const { return storage_; }
    const std::string& get_gpu() const { return gpu_; }
    bool has_wifi() const { return has_wifi_; }
    bool has_bluetooth() const { return has_bluetooth_; }
    const std::vector<std::string>& get_peripherals() const { return peripherals_; }

    // Метод для красивого вывода конфигурации
    void print_specs() const {
        std::cout << "\n🖥️  Конфигурация компьютера:" << std::endl;
        std::cout << "   ┌─────────────────────────────┐" << std::endl;
        std::cout << "   │ CPU:     " << std::left << std::setw(28) << cpu_ << "│" << std::endl;
        std::cout << "   │ RAM:     " << std::left << std::setw(28) << ram_ << "│" << std::endl;
        std::cout << "   │ Storage: " << std::left << std::setw(28) << storage_ << "│" << std::endl;
        std::cout << "   │ GPU:     " << std::left << std::setw(28) << (gpu_.empty() ? "Интегрированная" : gpu_) << "│" << std::endl;
        std::cout << "   │ WiFi:    " << std::left << std::setw(28) << (has_wifi_ ? "✅ Да" : "❌ Нет") << "│" << std::endl;
        std::cout << "   │ Bluetooth:" << std::left << std::setw(27) << (has_bluetooth_ ? "✅ Да" : "❌ Нет") << "│" << std::endl;
        std::cout << "   ├─────────────────────────────┤" << std::endl;
        std::cout << "   │ Периферия:                  │" << std::endl;
        if (peripherals_.empty()) {
            std::cout << "   │   (нет)                     │" << std::endl;
        } else {
            for (const auto& p : peripherals_) {
                std::cout << "   │   • " << std::left << std::setw(24) << p << "│" << std::endl;
            }
        }
        std::cout << "   └─────────────────────────────┘" << std::endl;
    }
};

// ============================================
// 🔨 АБСТРАКТНЫЙ СТРОИТЕЛЬ (интерфейс)
// ============================================
// Определяет шаги для создания продукта
class ComputerBuilder {
protected:
    // Строитель хранит экземпляр продукта
    std::unique_ptr<Computer> computer_;

public:
    ComputerBuilder() { computer_ = std::make_unique<Computer>(); }
    virtual ~ComputerBuilder() = default;

    // 🔑 Получение готового продукта
    std::unique_ptr<Computer> get_product() {
        return std::move(computer_);
    }

    // === Абстрактные шаги сборки ===
    virtual void build_cpu() = 0;
    virtual void build_ram() = 0;
    virtual void build_storage() = 0;
    virtual void build_gpu() = 0;
    virtual void build_connectivity() = 0;
    virtual void build_peripherals() = 0;

    // === Метод для полной сборки (шаблонный метод) ===
    void build_full() {
        build_cpu();
        build_ram();
        build_storage();
        build_gpu();
        build_connectivity();
        build_peripherals();
    }
};

// ============================================
// 🎮 КОНКРЕТНЫЙ СТРОИТЕЛЬ: Gaming PC
// ============================================
class GamingPCBuilder : public ComputerBuilder {
public:
    void build_cpu() override {
        computer_->set_cpu("Intel Core i9-14900K");
    }

    void build_ram() override {
        computer_->set_ram("32GB DDR5-6000MHz");
    }

    void build_storage() override {
        computer_->set_storage("2TB NVMe SSD + 4TB HDD");
    }

    void build_gpu() override {
        computer_->set_gpu("NVIDIA RTX 4090 24GB");
    }

    void build_connectivity() override {
        computer_->set_wifi(true);
        computer_->set_bluetooth(true);
    }

    void build_peripherals() override {
        computer_->add_peripheral("Mechanical Keyboard RGB");
        computer_->add_peripheral("Gaming Mouse 16000 DPI");
        computer_->add_peripheral("7.1 Surround Headset");
    }
};

// ============================================
// 💼 КОНКРЕТНЫЙ СТРОИТЕЛЬ: Office PC
// ============================================
class OfficePCBuilder : public ComputerBuilder {
public:
    void build_cpu() override {
        computer_->set_cpu("Intel Core i5-13400");
    }

    void build_ram() override {
        computer_->set_ram("16GB DDR4-3200MHz");
    }

    void build_storage() override {
        computer_->set_storage("512GB NVMe SSD");
    }

    void build_gpu() override {
        // Оставляем пустым — используем интегрированную графику
        computer_->set_gpu("");
    }

    void build_connectivity() override {
        computer_->set_wifi(true);
        computer_->set_bluetooth(false);  // Экономим
    }

    void build_peripherals() override {
        computer_->add_peripheral("Standard Keyboard");
        computer_->add_peripheral("Optical Mouse");
    }
};

// ============================================
// 🎨 КОНКРЕТНЫЙ СТРОИТЕЛЬ: Media PC (для монтажа)
// ============================================
class MediaPCBuilder : public ComputerBuilder {
public:
    void build_cpu() override {
        computer_->set_cpu("AMD Ryzen 9 7950X");
    }

    void build_ram() override {
        computer_->set_ram("64GB DDR5-5600MHz");  // Много RAM для рендеринга
    }

    void build_storage() override {
        computer_->set_storage("4TB NVMe SSD RAID");
    }

    void build_gpu() override {
        computer_->set_gpu("NVIDIA RTX 4080 16GB");
    }

    void build_connectivity() override {
        computer_->set_wifi(true);
        computer_->set_bluetooth(true);
    }

    void build_peripherals() override {
        computer_->add_peripheral("4K Color-Accurate Monitor");
        computer_->add_peripheral("Graphics Tablet");
        computer_->add_peripheral("Professional Audio Interface");
    }
};

// ============================================
// 👷 ДИРЕКТОР (Director) — управляет процессом сборки
// ============================================
// Не знает о конкретных билдерах, работает через интерфейс
class Director {
public:
    // 🔑 Сборка через любого строителя
    static std::unique_ptr<Computer> construct(ComputerBuilder& builder) {
        builder.build_full();  // Выполняем все шаги
        return builder.get_product();
    }

    // 🔑 Гибкая сборка: только нужные шаги
    static std::unique_ptr<Computer> construct_minimal(ComputerBuilder& builder) {
        builder.build_cpu();
        builder.build_ram();
        builder.build_storage();
        // Пропускаем GPU, connectivity, peripherals
        return builder.get_product();
    }
};

// ============================================
// 🧪 КЛИЕНТСКИЙ КОД: Демонстрация
// ============================================
void demonstrate_build(const std::string& title, ComputerBuilder& builder) {
    std::cout << "\n🔹 " << title << std::endl;
    std::cout << "   Процесс сборки..." << std::endl;

    // Директор управляет сборкой
    auto computer = Director::construct(builder);

    // Показываем результат
    computer->print_specs();
}

// ============================================
// 🚀 ТОЧКА ВХОДА
// ============================================
int main() {
    std::cout << "🔨 Builder Pattern Demo: Сборка компьютеров" << std::endl;
    std::cout << "============================================" << std::endl;

    // ────────────────────────────────────────
    // Сценарий 1: Игровой компьютер
    // ────────────────────────────────────────
    GamingPCBuilder gaming_builder;
    demonstrate_build("СБОРКА: Gaming PC", gaming_builder);

    // ────────────────────────────────────────
    // Сценарий 2: Офисный компьютер
    // ────────────────────────────────────────
    OfficePCBuilder office_builder;
    demonstrate_build("СБОРКА: Office PC", office_builder);

    // ────────────────────────────────────────
    // Сценарий 3: Компьютер для монтажа
    // ────────────────────────────────────────
    MediaPCBuilder media_builder;
    demonstrate_build("СБОРКА: Media PC", media_builder);

    // ────────────────────────────────────────
    // Сценарий 4: Гибкая сборка (только основы)
    // ────────────────────────────────────────
    std::cout << "\n🔹 ГИБКАЯ СБОРКА: Minimal PC" << std::endl;
    std::cout << "   Только CPU + RAM + Storage..." << std::endl;
    OfficePCBuilder minimal_builder;
    auto minimal_pc = Director::construct_minimal(minimal_builder);
    minimal_pc->print_specs();

    std::cout << "\n✅ Все компьютеры собраны!" << std::endl;
    return 0;
}

Writing builder.cpp


In [ ]:
# Компиляция с предупреждениями
!g++ -std=c++17 -Wall -Wextra -o builder builder.cpp

# Запуск программы
!./builder

🔨 Builder Pattern Demo: Сборка компьютеров

🔹 СБОРКА: Gaming PC
   Процесс сборки...

🖥️  Конфигурация компьютера:
   ┌─────────────────────────────┐
   │ CPU:     Intel Core i9-14900K        │
   │ RAM:     32GB DDR5-6000MHz           │
   │ Storage: 2TB NVMe SSD + 4TB HDD      │
   │ GPU:     NVIDIA RTX 4090 24GB        │
   │ WiFi:    ✅ Да                    │
   │ Bluetooth:✅ Да                   │
   ├─────────────────────────────┤
   │ Периферия:                  │
   │   • Mechanical Keyboard RGB │
   │   • Gaming Mouse 16000 DPI  │
   │   • 7.1 Surround Headset    │
   └─────────────────────────────┘

🔹 СБОРКА: Office PC
   Процесс сборки...

🖥️  Конфигурация компьютера:
   ┌─────────────────────────────┐
   │ CPU:     Intel Core i5-13400         │
   │ RAM:     16GB DDR4-3200MHz           │
   │ Storage: 512GB NVMe SSD              │
   │ GPU:     Интегрированная│
   │ WiFi:    ✅ Да                    │
   │ Bluetooth:❌ Нет                 │
   ├─────────────────────────────┤


# Прототип (Prototype)

Порождающий паттерн, который позволяет копировать существующие объекты без зависимости от их конкретных классов.

позволяет создавать новые объекты путём КОПИРОВАНИЯ существующих объектов-прототипов, а не через создание с нуля через конструктор.
🔑 Ключевая идея: Клонирование вместо конструирования

❌ Дорогое создание с нуля:
auto doc = new Document("complex initialization...");  // 500ms

✅ Быстрое клонирование:
auto doc_copy = doc->clone();  // 5ms

позволяет копировать существующие объекты, не делая ваш код зависимым от их классов.🔑 Ключевая идея: Клонирование вместо new!

🎮 Игры: Клонирование врагов, предметов
📊 Графика: Дублирование объектов сцены
📝 Документы: Шаблоны документов
👥 Пользователи: Копирование профилей с правами

In [ ]:
%%writefile prototype.cpp

#include <iostream>
#include <memory>
#include <string>
#include <vector>
#include <map>
#include <chrono>
#include <thread>

// ============================================
// АБСТРАКТНЫЙ ПРОТОТИП
// ============================================
class Prototype {
public:
    virtual ~Prototype() = default;
    virtual std::unique_ptr<Prototype> clone() const = 0;
    virtual std::string getName() const = 0;
};

// ============================================
// КОНКРЕТНЫЙ ПРОТОТИП: Документ
// ============================================
class Document : public Prototype {
private:
    std::string name_;
    std::string content_;
    std::vector<std::string> tags_;

public:
    Document(const std::string& name, const std::string& content)
        : name_(name), content_(content) {
        // Имитация дорогой операции
        std::this_thread::sleep_for(std::chrono::milliseconds(50));
    }

    std::unique_ptr<Prototype> clone() const override {
        return std::make_unique<Document>(*this);
    }

    std::string getName() const override {
        return name_;
    }

    void addTag(const std::string& tag) {
        tags_.push_back(tag);
    }

    void print() const {
        std::cout << "\n📄 Документ: " << name_ << std::endl;
        std::cout << "   Контент: " << content_ << std::endl;
        std::cout << "   Теги: ";
        for (size_t i = 0; i < tags_.size(); ++i) {
            std::cout << tags_[i];
            if (i < tags_.size() - 1) std::cout << ", ";
        }
        std::cout << std::endl;
    }
};

// ============================================
// КОНКРЕТНЫЙ ПРОТОТИП: Фигура
// ============================================
class Shape : public Prototype {
private:
    std::string name_;
    int x_, y_;
    std::string color_;

public:
    Shape(const std::string& name, int x, int y, const std::string& color)
        : name_(name), x_(x), y_(y), color_(color) {}

    std::unique_ptr<Prototype> clone() const override {
        auto cloned = std::make_unique<Shape>(*this);
        cloned->x_ += 10;
        cloned->y_ += 10;
        return cloned;
    }

    std::string getName() const override {
        return name_;
    }

    void print() const {
        std::cout << "\n🎨 Фигура: " << name_ << std::endl;
        std::cout << "   Позиция: (" << x_ << ", " << y_ << ")" << std::endl;
        std::cout << "   Цвет: " << color_ << std::endl;
    }
};

// ============================================
// РЕЕСТР ПРОТОТИПОВ
// ============================================
class PrototypeRegistry {
private:
    std::map<std::string, std::unique_ptr<Prototype>> cache_;

public:
    void setPrototype(const std::string& key, std::unique_ptr<Prototype> prototype) {
        cache_[key] = std::move(prototype);
    }

    std::unique_ptr<Prototype> getPrototype(const std::string& key) const {
        auto it = cache_.find(key);
        if (it != cache_.end()) {
            return it->second->clone();
        }
        return nullptr;
    }
};

// ============================================
// КЛИЕНТСКИЙ КОД
// ============================================
void clientCode(const Prototype& prototype) {
    std::cout << "\n--- Клонирование объекта ---" << std::endl;
    std::cout << "Оригинал: " << prototype.getName() << std::endl;

    auto clone = prototype.clone();
    std::cout << "Клон: " << clone->getName() << std::endl;
}

// ============================================
// MAIN
// ============================================
int main() {
    std::cout << "📋 Prototype Pattern" << std::endl;
    std::cout << "========================================" << std::endl;

    // ────────────────────────────────────────
    // Пример 1: Клонирование документа
    // ────────────────────────────────────────
    std::cout << "\n🔹 Пример 1: Документы" << std::endl;

    auto start = std::chrono::high_resolution_clock::now();
    auto doc1 = std::make_unique<Document>("Отчёт 2026", "Финансовые данные");
    doc1->addTag("финансы");
    doc1->addTag("2026");
    auto end = std::chrono::high_resolution_clock::now();
    auto create_time = std::chrono::duration_cast<std::chrono::milliseconds>(end - start).count();

    doc1->print();
    std::cout << "⏱️ Время создания: " << create_time << "ms" << std::endl;

    start = std::chrono::high_resolution_clock::now();
    auto doc2 = std::unique_ptr<Document>(dynamic_cast<Document*>(doc1->clone().release()));
    doc2->addTag("копия");
    end = std::chrono::high_resolution_clock::now();
    auto clone_time = std::chrono::duration_cast<std::chrono::milliseconds>(end - start).count();

    doc2->print();
    std::cout << "⏱️ Время клонирования: " << clone_time << "ms" << std::endl;
    std::cout << "📈 Выгода: " << (create_time > 0 ? create_time / (clone_time + 1) : 0) << "x быстрее" << std::endl;

    // ────────────────────────────────────────
    // Пример 2: Клонирование фигур
    // ────────────────────────────────────────
    std::cout << "\n🔹 Пример 2: Фигуры" << std::endl;

    auto shape1 = std::make_unique<Shape>("Круг", 0, 0, "Красный");
    shape1->print();

    std::cout << "\nКлонированные фигуры:" << std::endl;
    for (int i = 1; i <= 3; ++i) {
        auto shapeClone = std::unique_ptr<Shape>(dynamic_cast<Shape*>(shape1->clone().release()));
        std::cout << "Клон #" << i << ":" << std::endl;
        shapeClone->print();
    }

    // ────────────────────────────────────────
    // Пример 3: Реестр прототипов
    // ────────────────────────────────────────
    std::cout << "\n🔹 Пример 3: Реестр прототипов" << std::endl;

    PrototypeRegistry registry;
    registry.setPrototype("doc", std::make_unique<Document>("Шаблон", "Пустой"));
    registry.setPrototype("shape", std::make_unique<Shape>("Квадрат", 0, 0, "Синий"));

    auto docFromRegistry = std::unique_ptr<Document>(dynamic_cast<Document*>(registry.getPrototype("doc").release()));
    docFromRegistry->print();

    auto shapeFromRegistry = std::unique_ptr<Shape>(dynamic_cast<Shape*>(registry.getPrototype("shape").release()));
    shapeFromRegistry->print();

    std::cout << "\n✅ Prototype Pattern завершён!" << std::endl;
    return 0;
}

Overwriting prototype.cpp


In [ ]:
!g++ -std=c++17 -Wall -Wextra -o prototype prototype.cpp
!./prototype

📋 Prototype Pattern

🔹 Пример 1: Документы

📄 Документ: Отчёт 2026
   Контент: Финансовые данные
   Теги: финансы, 2026
⏱️ Время создания: 50ms

📄 Документ: Отчёт 2026
   Контент: Финансовые данные
   Теги: финансы, 2026, копия
⏱️ Время клонирования: 0ms
📈 Выгода: 50x быстрее

🔹 Пример 2: Фигуры

🎨 Фигура: Круг
   Позиция: (0, 0)
   Цвет: Красный

Клонированные фигуры:
Клон #1:

🎨 Фигура: Круг
   Позиция: (10, 10)
   Цвет: Красный
Клон #2:

🎨 Фигура: Круг
   Позиция: (10, 10)
   Цвет: Красный
Клон #3:

🎨 Фигура: Круг
   Позиция: (10, 10)
   Цвет: Красный

🔹 Пример 3: Реестр прототипов

📄 Документ: Шаблон
   Контент: Пустой
   Теги: 

🎨 Фигура: Квадрат
   Позиция: (10, 10)
   Цвет: Синий

✅ Prototype Pattern завершён!


#Singleton (Одиночка)

Порождающий паттерн, который гарантирует, что у класса есть только один экземпляр, и предоставляет глобальную точку доступа к нему.
Singleton гарантирует, что у класса только ОДИН экземпляр, и предоставляет глобальную точку доступа к нему.

🔑 Ключевая идея: Один объект на всё приложение!

📊 База данных: Один connection pool
📝 Логгер: Один лог-файл на приложение
⚙️ Конфигурация: Один объект настроек
🔐 Кэш: Единый кэш для всего приложения

In [ ]:
%%writefile singleton.cpp

#include <iostream>
#include <string>
#include <memory>
#include <mutex>
#include <thread>
#include <chrono>

// ============================================
// ОДИНОЧКА (THREAD-SAFE)
// ============================================
class Singleton {
private:
    // Статический указатель на единственный экземпляр
    static std::unique_ptr<Singleton> instance_;

    // Мьютекс для потокобезопасности
    static std::mutex mutex_;

    // Данные одиночки
    std::string value_;

    // Приватный конструктор — нельзя создать извне
    Singleton(const std::string& value) : value_(value) {
        std::this_thread::sleep_for(std::chrono::milliseconds(10));
    }

public:
    // Запрет копирования и присваивания
    Singleton(const Singleton&) = delete;
    Singleton& operator=(const Singleton&) = delete;

    // Деструктор
    ~Singleton() = default;

    // ========================================
    // ГЛОБАЛЬНАЯ ТОЧКА ДОСТУПА
    // ========================================
    static std::unique_ptr<Singleton>& getInstance() {
        std::lock_guard<std::mutex> lock(mutex_);
        if (!instance_) {
            instance_ = std::unique_ptr<Singleton>(new Singleton("Initial Value"));
        }
        return instance_;
    }

    // Методы для работы с данными
    std::string getValue() const {
        return value_;
    }

    void setValue(const std::string& value) {
        value_ = value;
    }

    void printInfo() const {
        std::cout << "🔒 Singleton: " << value_ << std::endl;
        std::cout << "   Адрес: " << this << std::endl;
    }
};

// Инициализация статических членов
std::unique_ptr<Singleton> Singleton::instance_ = nullptr;
std::mutex Singleton::mutex_;

// ============================================
// ОДИНОЧКА (MEYER'S SINGLETON — C++11)
// ============================================
// Более простой и безопасный вариант (локальная статическая переменная)
class ModernSingleton {
private:
    std::string value_;

    ModernSingleton(const std::string& value) : value_(value) {
        std::this_thread::sleep_for(std::chrono::milliseconds(10));
    }

public:
    // Запрет копирования
    ModernSingleton(const ModernSingleton&) = delete;
    ModernSingleton& operator=(const ModernSingleton&) = delete;

    // Глобальная точка доступа (C++11 гарантирует потокобезопасность)
    static ModernSingleton& getInstance() {
        static ModernSingleton instance("Modern Singleton");
        return instance;
    }

    std::string getValue() const { return value_; }
    void setValue(const std::string& value) { value_ = value; }

    void printInfo() const {
        std::cout << "🔒 ModernSingleton: " << value_ << std::endl;
        std::cout << "   Адрес: " << this << std::endl;
    }
};

// ============================================
// ТЕСТОВАЯ ФУНКЦИЯ ДЛЯ МНОГОПОТОЧНОСТИ
// ============================================
void testThread(int id) {
    std::cout << "🧵 Поток #" << id << " запускается..." << std::endl;

    auto& singleton = *Singleton::getInstance();
    std::cout << "   Поток #" << id << " получил Singleton по адресу: " << &singleton << std::endl;

    std::this_thread::sleep_for(std::chrono::milliseconds(50));
}

// ============================================
// MAIN
// ============================================
int main() {
    std::cout << "🔒 Singleton Pattern" << std::endl;
    std::cout << "========================================" << std::endl;

    // ────────────────────────────────────────
    // Пример 1: Базовое использование
    // ────────────────────────────────────────
    std::cout << "\n🔹 Пример 1: Базовое использование" << std::endl;

    auto& singleton1 = *Singleton::getInstance();
    singleton1.printInfo();

    std::cout << "\nИзменение значения..." << std::endl;
    singleton1.setValue("Updated Value");
    singleton1.printInfo();

    // Получаем второй раз — должен быть тот же объект!
    auto& singleton2 = *Singleton::getInstance();
    std::cout << "\nПовторное получение:" << std::endl;
    singleton2.printInfo();

    std::cout << "\n✅ Это один и тот же объект: " << (&singleton1 == &singleton2 ? "ДА" : "НЕТ") << std::endl;

    // ────────────────────────────────────────
    // Пример 2: Modern Singleton (C++11)
    // ────────────────────────────────────────
    std::cout << "\n🔹 Пример 2: Modern Singleton (C++11)" << std::endl;

    auto& modern1 = ModernSingleton::getInstance();
    modern1.printInfo();

    auto& modern2 = ModernSingleton::getInstance();
    std::cout << "\n✅ Это один и тот же объект: " << (&modern1 == &modern2 ? "ДА" : "НЕТ") << std::endl;

    // ────────────────────────────────────────
    // Пример 3: Многопоточный тест
    // ────────────────────────────────────────
    std::cout << "\n🔹 Пример 3: Многопоточный тест" << std::endl;

    std::thread threads[5];
    for (int i = 0; i < 5; ++i) {
        threads[i] = std::thread(testThread, i + 1);
    }

    for (int i = 0; i < 5; ++i) {
        threads[i].join();
    }

    std::cout << "\n✅ Все потоки получили ОДИН экземпляр!" << std::endl;

    // ────────────────────────────────────────
    // Пример 4: Проверка запрета копирования
    // ────────────────────────────────────────
    std::cout << "\n🔹 Пример 4: Запрет копирования" << std::endl;
    std::cout << "Следующий код НЕ скомпилируется (закомментирован):" << std::endl;
    std::cout << "   // Singleton copy = *Singleton::getInstance();  // ❌ Ошибка!" << std::endl;
    std::cout << "   // Singleton* ptr = new Singleton();            // ❌ Ошибка!" << std::endl;
    std::cout << "✅ Конструктор и копирование запрещены!" << std::endl;

    std::cout << "\n✅ Singleton Pattern завершён!" << std::endl;
    return 0;
}

Overwriting singleton.cpp


In [ ]:
!g++ -std=c++17 -Wall -Wextra -pthread -o singleton singleton.cpp
!./singleton

🔒 Singleton Pattern

🔹 Пример 1: Базовое использование
🔒 Singleton: Initial Value
   Адрес: 0x5af1f55ff2c0

Изменение значения...
🔒 Singleton: Updated Value
   Адрес: 0x5af1f55ff2c0

Повторное получение:
🔒 Singleton: Updated Value
   Адрес: 0x5af1f55ff2c0

✅ Это один и тот же объект: ДА

🔹 Пример 2: Modern Singleton (C++11)
🔒 ModernSingleton: Modern Singleton
   Адрес: 0x5af1c7f1a1c0

✅ Это один и тот же объект: ДА

🔹 Пример 3: Многопоточный тест
🧵 Поток #1 запускается...
   Поток #1 получил Singleton по адресу: 0x5af1f55ff2c0
🧵 Поток #2 запускается...
   Поток #2 получил Singleton по адресу: 0x5af1f55ff2c0
🧵 Поток #3 запускается...
   Поток #3 получил Singleton по адресу: 0x5af1f55ff2c0
🧵 Поток #4 запускается...
   Поток #4 получил Singleton по адресу: 0x5af1f55ff2c0
🧵 Поток #5 запускается...
   Поток #5 получил Singleton по адресу: 0x5af1f55ff2c0

✅ Все потоки получили ОДИН экземпляр!

🔹 Пример 4: Запрет копирования
Следующий код НЕ скомпилируется (закомментирован):
   // Singleton c

# Adapter (Адаптер)

Структурный паттерн, который позволяет объектам с несовместимыми интерфейсами работать вместе.

Adapter позволяет объектам с несовместимыми интерфейсами работать вместе, преобразуя один интерфейс в другой.

🔑 Ключевая идея: Преобразование интерфейсов!   

🔌 Электрические адаптеры (розетки)
📡 API интеграция (REST -> SOAP)
📊 Форматы данных (JSON -> XML)
🎮 Игровые контроллеры (Xbox -> PC)
📱 Драйверы устройств (USB -> Serial)

1. Target—интерфейс, который нужен клиенту
2. Adaptee—существующий класс с несовместимым интерфейс
3. Adapter— преобразует Adaptee в Target
4. Client— работает с Target, не зная об Adapter
📌 Adapter = Переводчик между несовместимыми интерфейсами!

In [ ]:
%%writefile adapter.cpp

#include <iostream>
#include <string>
#include <vector>
#include <memory>

// ============================================
// ЦЕЛЕВОЙ ИНТЕРФЕЙС (Target)
// ============================================
// Интерфейс, с которым работает клиентский код
class Target {
public:
    virtual ~Target() = default;
    virtual std::string request() const {
        return "Target: Базовое поведение";
    }
};

// ============================================
// АДАПТИРУЕМЫЙ КЛАСС (Adaptee)
// ============================================
// Класс с несовместимым интерфейсом
class Adaptee {
public:
    std::string specificRequest() const {
        return "Adaptee: Специфическое поведение";
    }
};

// ============================================
// АДАПТЕР (Adapter)
// ============================================
// Преобразует интерфейс Adaptee в интерфейс Target
class Adapter : public Target {
private:
    std::unique_ptr<Adaptee> adaptee_;

public:
    Adapter(std::unique_ptr<Adaptee> adaptee)
        : adaptee_(std::move(adaptee)) {}

    std::string request() const override {
        std::string result = adaptee_->specificRequest();
        // Преобразуем результат в формат Target
        return "Adapter: " + result;
    }
};

// ============================================
// КЛИЕНТСКИЙ КОД
// ============================================
// Работает только с интерфейсом Target
void clientCode(const Target& target) {
    std::cout << target.request() << std::endl;
}

// ============================================
// ПРИМЕР 2: Розетки и устройства
// ============================================

// Европейская розетка (Target)
class EUSocket {
public:
    virtual ~EUSocket() = default;
    virtual void plugIn() const = 0;
};

// Американская вилка (Adaptee)
class USPlug {
public:
    void insert() const {
        std::cout << "🇺🇸 Вилка США вставлена (110V)" << std::endl;
    }
};

// Адаптер для США -> ЕС
class UStoEUAdapter : public EUSocket {
private:
    USPlug usPlug_;

public:
    void plugIn() const override {
        std::cout << "🔌 Адаптер США->ЕС подключён" << std::endl;
        usPlug_.insert();
        std::cout << "⚡ Преобразование: 110V -> 220V" << std::endl;
    }
};

// Британская вилка (Adaptee)
class UKPlug {
public:
    void insert() const {
        std::cout << "🇬🇧 Вилка Великобритании вставлена (230V)" << std::endl;
    }
};

// Адаптер для Великобритании -> ЕС
class UKtoEUAdapter : public EUSocket {
private:
    UKPlug ukPlug_;

public:
    void plugIn() const override {
        std::cout << "🔌 Адаптер UK->ЕС подключён" << std::endl;
        ukPlug_.insert();
        std::cout << "⚡ Преобразование: 230V -> 220V" << std::endl;
    }
};

// ============================================
// ПРИМЕР 3: Работа с данными (JSON -> XML)
// ============================================

// Целевой интерфейс для работы с данными
class DataProcessor {
public:
    virtual ~DataProcessor() = default;
    virtual std::string processData(const std::string& data) const = 0;
};

// Существующий класс работает с JSON
class JSONProcessor {
public:
    std::string processJSON(const std::string& json) const {
        return "📄 JSON обработан: " + json;
    }
};

// Адаптер для использования JSONProcessor как DataProcessor
class JSONAdapter : public DataProcessor {
private:
    JSONProcessor jsonProcessor_;

public:
    std::string processData(const std::string& data) const override {
        // Преобразуем вызов в формат JSONProcessor
        return jsonProcessor_.processJSON(data);
    }
};

// ============================================
// MAIN
// ============================================
int main() {
    std::cout << "🔌 Adapter Pattern (refactoring.guru)" << std::endl;
    std::cout << "======================================" << std::endl;

    // ────────────────────────────────────────
    // Пример 1: Базовый адаптер
    // ────────────────────────────────────────
    std::cout << "\n🔹 Пример 1: Базовый адаптер" << std::endl;

    std::cout << "\nКлиент работает с Target напрямую:" << std::endl;
    Target target;
    clientCode(target);

    std::cout << "\nКлиент работает с Adaptee через Adapter:" << std::endl;
    auto adaptee = std::make_unique<Adaptee>();
    Adapter adapter(std::move(adaptee));
    clientCode(adapter);

    // ────────────────────────────────────────
    // Пример 2: Розетки и адаптеры
    // ────────────────────────────────────────
    std::cout << "\n🔹 Пример 2: Розетки и адаптеры" << std::endl;

    std::cout << "\n🇪🇺 Европейское устройство (нативная розетка):" << std::endl;
    class EUSocketImpl : public EUSocket {
    public:
        void plugIn() const override {
            std::cout << "🇪🇺 Вилка ЕС вставлена (220V)" << std::endl;
        }
    };
    EUSocketImpl euSocket;
    euSocket.plugIn();

    std::cout << "\n🇺🇸 Американское устройство (через адаптер):" << std::endl;
    UStoEUAdapter usAdapter;
    usAdapter.plugIn();

    std::cout << "\n🇬🇧 Британское устройство (через адаптер):" << std::endl;
    UKtoEUAdapter ukAdapter;
    ukAdapter.plugIn();

    // ────────────────────────────────────────
    // Пример 3: Обработка данных
    // ────────────────────────────────────────
    std::cout << "\n🔹 Пример 3: Обработка данных (JSON -> DataProcessor)" << std::endl;

    JSONAdapter jsonAdapter;
    std::string result = jsonAdapter.processData("{\"name\": \"test\"}");
    std::cout << result << std::endl;

    // ────────────────────────────────────────
    // Пример 4: Демонстрация совместимости
    // ────────────────────────────────────────
    std::cout << "\n🔹 Пример 4: Проверка совместимости" << std::endl;

    std::cout << "\n✅ Target и Adapter имеют одинаковый интерфейс!" << std::endl;
    std::cout << "✅ Клиентский код не знает об Adapter!" << std::endl;
    std::cout << "✅ Adaptee не требует изменений!" << std::endl;

    std::cout << "\n✅ Adapter Pattern завершён!" << std::endl;
    return 0;
}

Writing adapter.cpp


In [ ]:
!g++ -std=c++17 -Wall -Wextra -o adapter adapter.cpp
!./adapter

🔌 Adapter Pattern (refactoring.guru)

🔹 Пример 1: Базовый адаптер

Клиент работает с Target напрямую:
Target: Базовое поведение

Клиент работает с Adaptee через Adapter:
Adapter: Adaptee: Специфическое поведение

🔹 Пример 2: Розетки и адаптеры

🇪🇺 Европейское устройство (нативная розетка):
🇪🇺 Вилка ЕС вставлена (220V)

🇺🇸 Американское устройство (через адаптер):
🔌 Адаптер США->ЕС подключён
🇺🇸 Вилка США вставлена (110V)
⚡ Преобразование: 110V -> 220V

🇬🇧 Британское устройство (через адаптер):
🔌 Адаптер UK->ЕС подключён
🇬🇧 Вилка Великобритании вставлена (230V)
⚡ Преобразование: 230V -> 220V

🔹 Пример 3: Обработка данных (JSON -> DataProcessor)
📄 JSON обработан: {"name": "test"}

🔹 Пример 4: Проверка совместимости

✅ Target и Adapter имеют одинаковый интерфейс!
✅ Клиентский код не знает об Adapter!
✅ Adaptee не требует изменений!

✅ Adapter Pattern завершён!


# Bridge (Мост)

Структурный паттерн, который разделяет один или несколько классов на две отдельные иерархии — абстракцию и реализацию, позволяя изменять их независимо.

Bridge разделяет абстракцию и реализацию, позволяя им изменяться независимо друг от друга.

🔑 Ключевая идея: Композиция вместо наследования!  
🎮 Графика: Фигуры + Рендереры (OpenGL, DirectX, Vulkan)
📱 UI: Кнопки + Платформы (Windows, Mac, Linux)
📡 Сообщения: Типы + Каналы (Email, SMS, Push)
💾 Хранение: Данные + БД (MySQL, PostgreSQL, MongoDB)

1. Abstraction— высокоуровневый контроль (фигуры)
2. Implementation — низкоуровневые детали (рендеры)
3. Композиция— Abstraction содержит Implementation
4. Независимость — меняй одну часть без другой
📌 Bridge = Раздели и властвуй!

In [ ]:
%%writefile bridge.cpp

#include <iostream>
#include <string>
#include <memory>
#include <vector>

// ============================================
// РЕАЛИЗАЦИЯ (Implementation)
// ============================================
// Интерфейс для всех реализаций
class RenderAPI {
public:
    virtual ~RenderAPI() = default;
    virtual void renderCircle(float x, float y, float radius) const = 0;
    virtual void renderSquare(float x, float y, float size) const = 0;
    virtual std::string getAPIName() const = 0;
};

// ============================================
// КОНКРЕТНАЯ РЕАЛИЗАЦИЯ: OpenGL
// ============================================
class OpenGLRenderer : public RenderAPI {
public:
    void renderCircle(float x, float y, float radius) const override {
        std::cout << "🎮 OpenGL: Рендеринг круга в (" << x << ", " << y
                  << ") радиус " << radius << std::endl;
    }

    void renderSquare(float x, float y, float size) const override {
        std::cout << "🎮 OpenGL: Рендеринг квадрата в (" << x << ", " << y
                  << ") размер " << size << std::endl;
    }

    std::string getAPIName() const override {
        return "OpenGL";
    }
};

// ============================================
// КОНКРЕТНАЯ РЕАЛИЗАЦИЯ: DirectX
// ============================================
class DirectXRenderer : public RenderAPI {
public:
    void renderCircle(float x, float y, float radius) const override {
        std::cout << "🎮 DirectX: Рендеринг круга в (" << x << ", " << y
                  << ") радиус " << radius << std::endl;
    }

    void renderSquare(float x, float y, float size) const override {
        std::cout << "🎮 DirectX: Рендеринг квадрата в (" << x << ", " << y
                  << ") размер " << size << std::endl;
    }

    std::string getAPIName() const override {
        return "DirectX";
    }
};

// ============================================
// КОНКРЕТНАЯ РЕАЛИЗАЦИЯ: Software
// ============================================
class SoftwareRenderer : public RenderAPI {
public:
    void renderCircle(float x, float y, float radius) const override {
        std::cout << "💻 Software: Рендеринг круга в (" << x << ", " << y
                  << ") радиус " << radius << std::endl;
    }

    void renderSquare(float x, float y, float size) const override {
        std::cout << "💻 Software: Рендеринг квадрата в (" << x << ", " << y
                  << ") размер " << size << std::endl;
    }

    std::string getAPIName() const override {
        return "Software";
    }
};

// ============================================
// АБСТРАКЦИЯ (Abstraction)
// ============================================
// Содержит ссылку на реализацию
class Shape {
protected:
    std::shared_ptr<RenderAPI> renderAPI_;

public:
    explicit Shape(std::shared_ptr<RenderAPI> renderAPI)
        : renderAPI_(std::move(renderAPI)) {}

    virtual ~Shape() = default;
    virtual void draw() const = 0;
    virtual void move(float x, float y) = 0;
    virtual std::string getName() const = 0;
};

// ============================================
// УТОЧНЁННАЯ АБСТРАКЦИЯ: Круг
// ============================================
class Circle : public Shape {
private:
    float x_, y_, radius_;

public:
    Circle(std::shared_ptr<RenderAPI> renderAPI, float x, float y, float radius)
        : Shape(renderAPI), x_(x), y_(y), radius_(radius) {}

    void draw() const override {
        std::cout << "⭕ Круг: ";
        renderAPI_->renderCircle(x_, y_, radius_);
    }

    void move(float x, float y) override {
        x_ = x;
        y_ = y;
        std::cout << "   Перемещён в (" << x_ << ", " << y_ << ")" << std::endl;
    }

    std::string getName() const override {
        return "Circle";
    }
};

// ============================================
// УТОЧНЁННАЯ АБСТРАКЦИЯ: Квадрат
// ============================================
class Square : public Shape {
private:
    float x_, y_, size_;

public:
    Square(std::shared_ptr<RenderAPI> renderAPI, float x, float y, float size)
        : Shape(renderAPI), x_(x), y_(y), size_(size) {}

    void draw() const override {
        std::cout << "⬛ Квадрат: ";
        renderAPI_->renderSquare(x_, y_, size_);
    }

    void move(float x, float y) override {
        x_ = x;
        y_ = y;
        std::cout << "   Перемещён в (" << x_ << ", " << y_ << ")" << std::endl;
    }

    std::string getName() const override {
        return "Square";
    }
};

// ============================================
// УТОЧНЁННАЯ АБСТРАКЦИЯ: Треугольник
// ============================================
class Triangle : public Shape {
private:
    float x_, y_, base_, height_;

public:
    Triangle(std::shared_ptr<RenderAPI> renderAPI, float x, float y, float base, float height)
        : Shape(renderAPI), x_(x), y_(y), base_(base), height_(height) {}

    void draw() const override {
        std::cout << "🔺 Треугольник: ";
        std::cout << "Рендеринг через " << renderAPI_->getAPIName() << std::endl;
        std::cout << "   Позиция: (" << x_ << ", " << y_ << ")" << std::endl;
        std::cout << "   Размер: основание " << base_ << ", высота " << height_ << std::endl;
    }

    void move(float x, float y) override {
        x_ = x;
        y_ = y;
        std::cout << "   Перемещён в (" << x_ << ", " << y_ << ")" << std::endl;
    }

    std::string getName() const override {
        return "Triangle";
    }
};

// ============================================
// КЛИЕНТСКИЙ КОД
// ============================================
void clientCode(const Shape& shape) {
    std::cout << "Фигура: " << shape.getName() << std::endl;
    shape.draw();
}

// ============================================
// MAIN
// ============================================
int main() {
    std::cout << "🌉 Bridge Pattern " << std::endl;
    std::cout << "=====================================" << std::endl;

    // ────────────────────────────────────────
    // Пример 1: Разные фигуры с OpenGL
    // ────────────────────────────────────────
    std::cout << "\n🔹 Пример 1: Фигуры с OpenGL" << std::endl;

    auto openGL = std::make_shared<OpenGLRenderer>();

    Circle circle1(openGL, 0.0f, 0.0f, 5.0f);
    Square square1(openGL, 10.0f, 10.0f, 8.0f);
    Triangle triangle1(openGL, 5.0f, 5.0f, 6.0f, 4.0f);

    clientCode(circle1);
    clientCode(square1);
    clientCode(triangle1);

    // ────────────────────────────────────────
    // Пример 2: Те же фигуры с DirectX
    // ────────────────────────────────────────
    std::cout << "\n🔹 Пример 2: Фигуры с DirectX" << std::endl;

    auto directX = std::make_shared<DirectXRenderer>();

    Circle circle2(directX, 0.0f, 0.0f, 5.0f);
    Square square2(directX, 10.0f, 10.0f, 8.0f);

    clientCode(circle2);
    clientCode(square2);

    // ────────────────────────────────────────
    // Пример 3: Динамическая смена реализации
    // ────────────────────────────────────────
    std::cout << "\n🔹 Пример 3: Смена реализации во время выполнения" << std::endl;

    auto software = std::make_shared<SoftwareRenderer>();
    Circle circle3(software, 0.0f, 0.0f, 5.0f);

    std::cout << "С SoftwareRenderer:" << std::endl;
    circle3.draw();

    // Меняем реализацию на лету!
    circle3 = Circle(openGL, 0.0f, 0.0f, 5.0f);
    std::cout << "\nС OpenGLRenderer:" << std::endl;
    circle3.draw();

    // ────────────────────────────────────────
    // Пример 4: Демонстрация независимости
    // ────────────────────────────────────────
    std::cout << "\n🔹 Пример 4: Проверка независимости иерархий" << std::endl;

    std::cout << "\n✅ Абстракции (фигуры): Circle, Square, Triangle" << std::endl;
    std::cout << "✅ Реализации (рендеры): OpenGL, DirectX, Software" << std::endl;
    std::cout << "✅ Можно комбинировать любые фигуры с любыми рендерами!" << std::endl;
    std::cout << "✅ Добавление новой фигуры не требует изменения рендеров!" << std::endl;
    std::cout << "✅ Добавление нового рендера не требует изменения фигур!" << std::endl;

    std::cout << "\n✅ Bridge Pattern завершён!" << std::endl;
    return 0;
}

Overwriting bridge.cpp


In [ ]:
!g++ -std=c++17 -Wall -Wextra -o bridge bridge.cpp
!./bridge

🌉 Bridge Pattern 

🔹 Пример 1: Фигуры с OpenGL
Фигура: Circle
⭕ Круг: 🎮 OpenGL: Рендеринг круга в (0, 0) радиус 5
Фигура: Square
⬛ Квадрат: 🎮 OpenGL: Рендеринг квадрата в (10, 10) размер 8
Фигура: Triangle
🔺 Треугольник: Рендеринг через OpenGL
   Позиция: (5, 5)
   Размер: основание 6, высота 4

🔹 Пример 2: Фигуры с DirectX
Фигура: Circle
⭕ Круг: 🎮 DirectX: Рендеринг круга в (0, 0) радиус 5
Фигура: Square
⬛ Квадрат: 🎮 DirectX: Рендеринг квадрата в (10, 10) размер 8

🔹 Пример 3: Смена реализации во время выполнения
С SoftwareRenderer:
⭕ Круг: 💻 Software: Рендеринг круга в (0, 0) радиус 5

С OpenGLRenderer:
⭕ Круг: 🎮 OpenGL: Рендеринг круга в (0, 0) радиус 5

🔹 Пример 4: Проверка независимости иерархий

✅ Абстракции (фигуры): Circle, Square, Triangle
✅ Реализации (рендеры): OpenGL, DirectX, Software
✅ Можно комбинировать любые фигуры с любыми рендерами!
✅ Добавление новой фигуры не требует изменения рендеров!
✅ Добавление нового рендера не требует изменения фигур!

✅ Bridge Pattern завер

#Composite (Компоновщик)


Структурный паттерн, который позволяет сгруппировать объекты в древовидную структуру и работать с ними как с единым объектом.


обеспечить единообразную обработку как простых объектов (листов), так и их композиций (контейнеров) через общий интерфейс. Это избавляет клиентский код от необходимости проверять тип объекта перед использованием, упрощая работу со сложными иерархиями.
Структура паттерна включает три основных участника. Component объявляет общий интерфейс для всех объектов в композиции, может определять методы доступа к потомкам. Leaf представляет простые объекты без потомков, реализует основную логику работы. Composite хранит дочерние компоненты и реализует методы управления ими (add, remove, getChild), делегируя операции потомкам при выполнении. Клиент работает со всеми объектами через интерфейс Component, не различая листья и контейнеры.

In [ ]:
%%writefile composite.cpp

#include <iostream>
#include <string>
#include <vector>
#include <memory>

class Component {
public:
    virtual ~Component() = default;
    virtual void operation() const = 0;
    virtual std::string getName() const = 0;
};

class Leaf : public Component {
private:
    std::string name_;
public:
    explicit Leaf(const std::string& name) : name_(name) {}
    void operation() const override {
        std::cout << "Leaf: " << name_ << std::endl;
    }
    std::string getName() const override { return name_; }
};

class Composite : public Component {
private:
    std::string name_;
    std::vector<std::shared_ptr<Component>> children_;
public:
    explicit Composite(const std::string& name) : name_(name) {}
    void operation() const override {
        std::cout << "Composite: " << name_ << std::endl;
        for (const auto& child : children_) {
            child->operation();
        }
    }
    void add(std::shared_ptr<Component> component) {
        children_.push_back(std::move(component));
    }
    std::string getName() const override { return name_; }
};

int main() {
    auto root = std::make_shared<Composite>("Root");
    root->add(std::make_shared<Leaf>("File1.txt"));
    root->add(std::make_shared<Leaf>("File2.txt"));
    root->operation();
    return 0;
}

Writing composite.cpp


In [ ]:
!g++ -std=c++17 -Wall -o composite composite.cpp

In [ ]:
!./composite

Composite: Root
Leaf: File1.txt
Leaf: File2.txt


In [ ]:
!cat composite.cpp


#include <iostream>
#include <string>
#include <vector>
#include <memory>

class Component {
public:
    virtual ~Component() = default;
    virtual void operation() const = 0;
    virtual std::string getName() const = 0;
};

class Leaf : public Component {
private:
    std::string name_;
public:
    explicit Leaf(const std::string& name) : name_(name) {}
    void operation() const override {
        std::cout << "Leaf: " << name_ << std::endl;
    }
    std::string getName() const override { return name_; }
};

class Composite : public Component {
private:
    std::string name_;
    std::vector<std::shared_ptr<Component>> children_;
public:
    explicit Composite(const std::string& name) : name_(name) {}
    void operation() const override {
        std::cout << "Composite: " << name_ << std::endl;
        for (const auto& child : children_) {
            child->operation();
        }
    }
    void add(std::shared_ptr<Component> component) {
        children_.push_back(std::move(com